In [1]:
!pip3 install transformers
!pip3 install --upgrade torch 

import sys
import os

module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from captions_dataset import CaptionsDataset

dataset_train = CaptionsDataset(data_path="../data", split="train")
dataset_val = CaptionsDataset(data_path="../data", split="val")

Requirement already up-to-date: torch in /home/lrinelli/.local/lib/python3.8/site-packages (1.8.1)
Dataset json file, loading dataset...
There are 118287 captions in total (train)
The following 91 categories are present in the dataset:
['bicycle', 'vehicle', 'clock', 'indoor', 'cup', 'kitchen', 'sink', 'appliance', 'vase', 'car', 'motorcycle', 'person', 'bench', 'outdoor', 'airplane', 'bottle', 'toilet', 'furniture', 'potted plant', 'toothbrush', 'refrigerator', 'oven', 'apple', 'food', 'banana', 'surfboard', 'sports', 'bowl', 'spoon', 'traffic light', 'cat', 'animal', 'cow', 'handbag', 'accessory', 'umbrella', 'book', 'skateboard', 'horse', 'cake', 'donut', 'chair', 'cell phone', 'electronic', 'laptop', 'mouse', 'tv', 'dog', 'knife', 'orange', 'microwave', 'backpack', 'bus', 'dining table', 'truck', 'bird', 'giraffe', 'suitcase', 'boat', 'skis', 'fire hydrant', 'toaster', 'wine glass', 'sheep', 'remote', 'scissors', 'tie', 'kite', 'broccoli', 'stop sign', 'teddy bear', 'fork', 'keyboa

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
print("Tokenizer before added special tokens "+str(len(tokenizer)))
special_tokens_dict = {'additional_special_tokens': dataset_train.control_codes}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print("added "+str(num_added_toks)+" tokens to the pretrained tokenizer")

# add the EOS token as PAD token to avoid warnings
model = GPT2LMHeadModel.from_pretrained("../data/pretrained")
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Tokenizer before added special tokens 50259
added 91 tokens to the pretrained tokenizer


Embedding(50350, 768)

In [24]:
import torch
prompt="<CTRL:indoor><|startoftext|> A person is"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

model.eval()
sample_outputs = model.generate(generated,do_sample=True,   
                                top_k=100, 
                                max_length = 256,
                                top_p=0.90, 
                                num_return_sequences=3)

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0:  A person is holding a bottle of red wine and looking at it.


1:  A person is standing on a beach looking in the distance.


2:  A person is riding a snow board on a snowy slope.


